In [ ]:
import pandas as pd
import json
import os
import numpy as np  # Importa numpy per utilizzare np.nan

def load_data(file_path):
    """
    Carica un file CSV, XLS, JSON o JSONL e restituisce un DataFrame con le colonne "name", "category", "address".
    """
    encodings = ['utf-8', 'ISO-8859-1', 'latin1']
    
    for encoding in encodings:
        try:
            if file_path.endswith('.csv'):
                df = pd.read_csv(file_path, encoding=encoding)
            elif file_path.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(file_path)
            elif file_path.endswith('.json'):
                with open(file_path, 'r', encoding=encoding) as f:
                    data = json.load(f)
                df = pd.DataFrame(data)
            elif file_path.endswith('.jsonl'):
                df = pd.read_json(file_path, lines=True)
            else:
                raise ValueError("Formato non supportato")
            
            # Definiamo i possibili nomi per "name", "category" e "address"
            name_columns = ["Name", "name", "Company", "BRAND NAME"]
            category_columns = ["category", "null", "Area of Activity", "company_business", "industry", "CATEGORY", "Industry"]
            address_columns = ["address", "Address Name", "Address"]
            location_columns = ["city", "headquarters_region_city", "headquarters", "location", "Headquarter"]

            # Inizializziamo name, category e address come NaN
            name_value = np.nan
            category_value = np.nan
            address_value = np.nan

            # Troviamo il primo campo esistente per "name"
            for col in name_columns:
                if col in df.columns:
                    name_value = df[col].astype(str).str.strip()  # Manteniamo i nomi esattamente come nel file
                    break

            # Troviamo il primo campo esistente per "category"
            for col in category_columns:
                if col in df.columns:
                    category_value = df[col]
                    break

            # Troviamo il primo campo esistente per "address" e "location"
            for col in address_columns:
                if col in df.columns:
                    address_value = df[col]
                    # Se la colonna è "address" tutto in minuscolo, prendi il terzo elemento separato dalle virgole
                    if col == "address":
                        address_value = address_value.str.split(',').str[2]  # Terzo elemento
                    # Se la colonna è "Headquarter" o "headquarters", prendi il primo elemento separato dalle virgole
                    elif col in ["Headquarter", "headquarters"]:
                        address_value = address_value.str.split(',').str[0]  # Primo elemento
                    break

            # Crea un DataFrame con le colonne "name", "category" e "address"
            result_df = pd.DataFrame({"name": name_value, "category": category_value, "address": address_value}, index=range(len(df)))
            return result_df
        
        except Exception as e:
            print(f"Errore con il file {file_path} usando encoding {encoding}: {e}")
    
    # In caso di errore dopo aver provato tutti gli encoding
    print(f"Impossibile caricare il file {file_path} con i codici di encoding disponibili.")
    return pd.DataFrame(columns=["name", "category", "address"])

# Cartella contenente i file
source_folder = "sources/"

# Ottieni la lista dei file nella cartella source
file_list = [os.path.join(source_folder, file) for file in os.listdir(source_folder) 
             if file.endswith(('.csv', '.xls', '.xlsx', '.json', '.jsonl'))]

# Separare i file in due gruppi
wissel_files = [file for file in file_list if os.path.basename(file) in [
    "wissel-rappresentanti-ariregister.rik.ee.csv", "wissel-partners-ariregister.rik.ee.csv"]]
other_files = [file for file in file_list if file not in wissel_files]

# Creare due DataFrame separati
wissel_df = pd.concat([load_data(file) for file in wissel_files], ignore_index=True)
other_df = pd.concat([load_data(file) for file in other_files], ignore_index=True)

# Salvare i risultati in due CSV separati
wissel_df.to_csv("output_wissel.csv", index=False, encoding='utf-8')
other_df.to_csv("output_other.csv", index=False, encoding='utf-8')

print("Schema unificato salvato in 'output_wissel.csv' e 'output_other.csv'")
